## Item-Item Top Ten Similar 

In [4]:
import pandas as pd
import nltk

from ast import literal_eval
# nltk.download('punkt')
# nltk.download('stopwords')

import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 10, 5

In [110]:
credit_df=pd.read_csv('../input/tmdb-movie-metadata/tmdb_5000_credits.csv')
credit_df.columns = ['id', 'title', 'cast', 'crew']
credit_df.describe()

movie_df=pd.read_csv('../input/tmdb-movie-metadata/tmdb_5000_movies.csv')
movie_df.columns
movie_df.describe()

,id
count,4803.000000
mean,57165.484281
std,88694.614033
min,5.000000
25%,9014.500000
50%,14629.000000
75%,58610.500000
max,459488.000000


Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,4.803000e+03,4803.000000,4803.000000,4.803000e+03,4801.000000,4803.000000,4803.000000
mean,2.904504e+07,57165.484281,21.492301,8.226064e+07,106.875859,6.092172,690.217989
std,4.072239e+07,88694.614033,31.816650,1.628571e+08,22.611935,1.194612,1234.585891
min,0.000000e+00,5.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,7.900000e+05,9014.500000,4.668070,0.000000e+00,94.000000,5.600000,54.000000
50%,1.500000e+07,14629.000000,12.921594,1.917000e+07,103.000000,6.200000,235.000000
75%,4.000000e+07,58610.500000,28.313505,9.291719e+07,118.000000,6.800000,737.000000
max,3.800000e+08,459488.000000,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


### Preprocess

In [111]:
movie_df = movie_df.merge(credit_df, on='id')
movie_df.columns

movie_df['overview'] = movie_df['overview'].fillna('')
movie_df.sample(1)

for key in ['cast', 'crew', 'keywords', 'genres']:
    movie_df[key] = movie_df[key].apply(literal_eval)
    
movie_df[['id','overview','cast', 'keywords', 'genres']].head(3)

def get_list_name(x):
    if isinstance(x, list):
        names = [i['name'] for i in x if len(i['name'])> 2]
        names = ' '.join(names)
        return names
    return []

for col in ['cast', 'keywords', 'genres']:
    movie_df[col+'_'] = movie_df[col].apply(get_list_name)
    
movie_df[['overview','cast', 'keywords', 'genres','cast_', 'keywords_', 'genres_']].head(3)

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title_x', 'vote_average',
       'vote_count', 'title_y', 'cast', 'crew'],
      dtype='object')

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title_x,vote_average,vote_count,title_y,cast,crew
737,60000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 18, ""nam...",NaN,137094,"[{""id"": 212, ""name"": ""london england""}, {""id"":...",en,Jack Ryan: Shadow Recruit,"Jack Ryan, as a young covert CIA analyst, unco...",42.538258,"[{""name"": ""Paramount Pictures"", ""id"": 4}, {""na...",...,105.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Trust no one.,Jack Ryan: Shadow Recruit,5.9,1173,Jack Ryan: Shadow Recruit,"[{""cast_id"": 14, ""character"": ""Jack Ryan"", ""cr...","[{""credit_id"": ""53902dad0e0a2643fc0007a5"", ""de..."


,id,overview,cast,keywords,genres
0,19995,"In the 22nd century, a paraplegic Marine is di...","[{'cast_id': 242, 'character': 'Jake Sully', '...","[{'id': 1463, 'name': 'culture clash'}, {'id':...","[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam..."
1,285,"Captain Barbossa, long believed to be dead, ha...","[{'cast_id': 4, 'character': 'Captain Jack Spa...","[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na...","[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,206647,A cryptic message from Bond’s past sends him o...,"[{'cast_id': 1, 'character': 'James Bond', 'cr...","[{'id': 470, 'name': 'spy'}, {'id': 818, 'name...","[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam..."


,overview,cast,keywords,genres,cast_,keywords_,genres_
0,"In the 22nd century, a paraplegic Marine is di...","[{'cast_id': 242, 'character': 'Jake Sully', '...","[{'id': 1463, 'name': 'culture clash'}, {'id':...","[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",Sam Worthington Zoe Saldana Sigourney Weaver S...,culture clash future space war space colony so...,Action Adventure Fantasy Science Fiction
1,"Captain Barbossa, long believed to be dead, ha...","[{'cast_id': 4, 'character': 'Captain Jack Spa...","[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na...","[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",Johnny Depp Orlando Bloom Keira Knightley Stel...,ocean drug abuse exotic island east india trad...,Adventure Fantasy Action
2,A cryptic message from Bond’s past sends him o...,"[{'cast_id': 1, 'character': 'James Bond', 'cr...","[{'id': 470, 'name': 'spy'}, {'id': 818, 'name...","[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",Daniel Craig Christoph Waltz Léa Seydoux Ralph...,spy based on novel secret agent sequel mi6 bri...,Action Adventure Crime


### Create Soup1: using keywords, cast and genres

In [121]:
def create_soup(x):
    return x.keywords_ + x.cast_ + x.genres_

movie_df['soup'] = movie_df.apply(create_soup, axis=1)
movie_df.soup.sample(2)

925     soulmates midlife crisis marriage crisis woman...
2618    1970s drums groupie musical rock heavy metal h...
Name: soup, dtype: object

### Create Soup2: using overview, keywords, cast and genres

In [120]:
def create_soup2(x):
    return x.overview + x.keywords_ + x.cast_ + x.genres_

movie_df['soup2'] = movie_df.apply(create_soup2, axis=1)
movie_df.soup.sample(2)

2982    Jigsaw and his apprentice Amanda are dead. Now...
4539    In the California desert, the adventures of a ...
Name: soup, dtype: object

### Similar Cosin use TfidfVectorizer 
* **lowercase:** default is convert to lowcase
* **analyzer:** default is tokenlize to words
* **stop_words:** use 'english'
* **ngram_range:** creat n-grame, use default compare (1,1) (1,2)
* **min_df:** cut-off in the literature, avoid low freq words
* **max_features:** limit word features, keep highest term frequency across

In [86]:
def create_tfidf(col, n_gram=(1,1)):
    from sklearn.feature_extraction.text import TfidfVectorizer

    #Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
    tfidf = TfidfVectorizer(stop_words='english', ngram_range=n_gram)

    #Construct the required TF-IDF matrix by fitting and transforming the data
    tfidf_matrix = tfidf.fit_transform(movie_df[col])
    
    print("TFIDF", tfidf_matrix.shape)
    return tfidf_matrix

#### Create cosine similarity matrix

In [85]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

def cosine_matrix(tf_idf):
    # Compute the cosine similarity matrix
    cosine_sim = linear_kernel(tf_idf, tf_idf)
    print("Similarity Shape", cosine_sim.shape)
    return cosine_sim

### Movie dictionary of Title

In [28]:
indices = pd.Series(movie_df.index, index=movie_df['title_x']).drop_duplicates()
indices

title_x
Avatar                                         0
Pirates of the Caribbean: At World's End       1
Spectre                                        2
The Dark Knight Rises                          3
John Carter                                    4
                                            ... 
El Mariachi                                 4798
Newlyweds                                   4799
Signed, Sealed, Delivered                   4800
Shanghai Calling                            4801
My Date with Drew                           4802
Length: 4803, dtype: int64

## Get recommend from looking up Matrix

In [91]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]
    print(idx)

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return movie_df['title_x'].iloc[movie_indices]

**Now we have enough gear to make some test**
### Test1 - using soup of keywords, cast and genres

In [92]:
tf_idf = create_tfidf('soup')
cosine_sim = cosine_matrix(tf_idf)
get_recommendations('The Dark Knight Rises', cosine_sim)
get_recommendations('Mission: Impossible', cosine_sim)

TFIDF (4803, 47903)
Similarity Shape (4803, 4803)
3


65              The Dark Knight
119               Batman Begins
298     The Wolf of Wall Street
16                 The Avengers
2207                  12 Rounds
95                 Interstellar
2041              Hail, Caesar!
81                   Maleficent
30                 Spider-Man 2
813                    Superman
Name: title_x, dtype: object

425


4339                             Dr. No
213              Mission: Impossible II
139             Mission: Impossible III
3143                You Only Live Twice
3251    On Her Majesty's Secret Service
3343                   Live and Let Die
1200               The Living Daylights
695                           GoldenEye
2398                             Hitman
282                           True Lies
Name: title_x, dtype: object

### Test2 - Using overview n-gram 1

In [93]:
tf_idf = create_tfidf('overview', (1,1))
cosine_sim = cosine_matrix(tf_idf)
get_recommendations('The Dark Knight Rises', cosine_sim)
get_recommendations('Mission: Impossible', cosine_sim)

TFIDF (4803, 20978)
Similarity Shape (4803, 4803)
3


65                              The Dark Knight
299                              Batman Forever
428                              Batman Returns
1359                                     Batman
3854    Batman: The Dark Knight Returns, Part 2
119                               Batman Begins
2507                                  Slow Burn
9            Batman v Superman: Dawn of Justice
1181                                        JFK
210                              Batman & Robin
Name: title_x, dtype: object

425


139                  Mission: Impossible III
213                   Mission: Impossible II
753                             The Sentinel
3943                               Compadres
1701              Once Upon a Time in Mexico
594                              Bad Company
153     Mission: Impossible - Ghost Protocol
4689               The Trials Of Darryl Hunt
884                         Zero Dark Thirty
973                                    Basic
Name: title_x, dtype: object

### Test3 - Using overview n-gram 2

In [103]:
tf_idf = create_tfidf('overview', (2,2))
cosine_sim = cosine_matrix(tf_idf)
get_recommendations('The Dark Knight Rises',cosine_sim)
get_recommendations('Mission: Impossible',cosine_sim)

TFIDF (4803, 115971)
Similarity Shape (4803, 4803)
3


299           Batman Forever
65           The Dark Knight
1359                  Batman
2507               Slow Burn
2632           Transsiberian
884         Zero Dark Thirty
1181                     JFK
2193    Secret in Their Eyes
4649         Shotgun Stories
879      Law Abiding Citizen
Name: title_x, dtype: object

425


389                         The Three Musketeers
139                      Mission: Impossible III
213                       Mission: Impossible II
153         Mission: Impossible - Ghost Protocol
3197                              Flame & Citron
0                                         Avatar
1       Pirates of the Caribbean: At World's End
2                                        Spectre
3                          The Dark Knight Rises
4                                    John Carter
Name: title_x, dtype: object

### Test4 - using soup2 of keywords, cast and genres with overview added

In [124]:
tf_idf = create_tfidf('soup2')
cosine_sim = cosine_matrix(tf_idf)
get_recommendations('The Dark Knight Rises', cosine_sim)
get_recommendations('Mission: Impossible', cosine_sim)

TFIDF (4803, 60739)
Similarity Shape (4803, 4803)
3


65                         The Dark Knight
119                          Batman Begins
428                         Batman Returns
1359                                Batman
299                         Batman Forever
298                The Wolf of Wall Street
210                         Batman & Robin
9       Batman v Superman: Dawn of Justice
16                            The Avengers
30                            Spider-Man 2
Name: title_x, dtype: object

425


213              Mission: Impossible II
139             Mission: Impossible III
594                         Bad Company
4339                             Dr. No
3143                You Only Live Twice
3343                   Live and Let Die
1200               The Living Daylights
3251    On Her Majesty's Secret Service
282                           True Lies
372                            Spy Game
Name: title_x, dtype: object

### Using keywords only

In [98]:
tf_idf = create_tfidf('keywords_', (2,2))
cosine_sim = cosine_matrix(tf_idf)
get_recommendations('The Dark Knight Rises', cosine_sim)
get_recommendations('Mission: Impossible', cosine_sim)

TFIDF (4803, 33617)
Similarity Shape (4803, 4803)
3


65           The Dark Knight
119            Batman Begins
428           Batman Returns
1359                  Batman
210           Batman & Robin
3658             The Cottage
2793    The Killer Inside Me
813                 Superman
1740              Kick-Ass 2
988                 Mad City
Name: title_x, dtype: object

425


3725                                The Sweeney
4339                                     Dr. No
219             Charlie's Angels: Full Throttle
3143                        You Only Live Twice
752                         My Favorite Martian
70                               Wild Wild West
1821           Scooby-Doo 2: Monsters Unleashed
578     Alvin and the Chipmunks: The Squeakquel
1301                                 Sgt. Bilko
3343                           Live and Let Die
Name: title_x, dtype: object

In [100]:
tf_idf = create_tfidf('title_x', (2,2))
get_recommendations('The Dark Knight Rises', cosine_sim)
get_recommendations('Mission: Impossible', cosine_sim)

TFIDF (4803, 4038)
3


65           The Dark Knight
119            Batman Begins
428           Batman Returns
1359                  Batman
210           Batman & Robin
3658             The Cottage
2793    The Killer Inside Me
813                 Superman
1740              Kick-Ass 2
988                 Mad City
Name: title_x, dtype: object

425


3725                                The Sweeney
4339                                     Dr. No
219             Charlie's Angels: Full Throttle
3143                        You Only Live Twice
752                         My Favorite Martian
70                               Wild Wild West
1821           Scooby-Doo 2: Monsters Unleashed
578     Alvin and the Chipmunks: The Squeakquel
1301                                 Sgt. Bilko
3343                           Live and Let Die
Name: title_x, dtype: object

## Conclution
* Seem N-gram 1 give better results
* Use overview seem better than soup of keywords, cast and genres
* Add overview to Soup not gave any significan result

**Overview choosen**

## Problems & Question
### Memory problem:
This is just demo data with only 5k record but cosine matrix already 180MB \
But implement of group must dealing with 48k records \
So the cosine matri is to big 48k^2*8 ~ 18GB \
Have to separate create TFIDF deploy then web server create matrix for only one movie of API request
### Question:
How if 
* create cosine similarity matrix from each of these 4 features: m1, m2, m3, m4
* Product matrix of that four matrix sum(m1,m2,m3,m4)
* product with weight: Msum = w1.m1 + w2.m2 + w3.m3 + w4.m4